In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import calendar

# Request

1. Options: https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?instrumentType=OPTIDX&symbol=BANKNIFTY&expiryDate=28-10-2021&optionType=CE&strikePrice=34000&dateRange=&fromDate=01-Sep-2021&toDate=28-Oct-2021&segmentLink=9&symbolCount=
2. Prices: https://www1.nseindia.com/products/dynaContent/equities/indices/historicalindices.jsp?indexType=NIFTY%20BANK&fromDate=01-06-2022&toDate=22-06-2022
3. Stock Options: https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?instrumentType=OPTSTK&symbol=BANKNIFTY&expiryDate=28-10-2021&optionType=CE&strikePrice=34000&dateRange=&fromDate=01-Sep-2021&toDate=28-Oct-2021&segmentLink=9&symbolCount=
4. Stock Prices: https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?symbol=ADANIENT&segmentLink=3&symbolCount=1&series=ALL&dateRange=+&fromDate=01-07-2022&toDate=02-07-2022&dataType=PRICEVOLUMEDELIVERABLE

In [14]:
class GetData:
    def __init__(self,from_date,to_date,prices_file):
        self.from_date = from_date
        self.to_date = to_date
        self.prices_file = prices_file
        self.headers = {
            "Accept":"*/*",
            "Referer":"https://www1.nseindia.com/products/content/derivatives/equities/historical_fo.htm",
            "Accept-Encoding":"gzip, deflate",
            "User-Agent":"Google",
            "Accept-Language":"en-US,en;q=0.9,hi;q=0.8,ru;q=0.7"
        }

        self.proxies = {
            "http":"127.0.0.1:8080",
            "https":"127.0.0.1:8080"
        }
        
        self.months = {
            1:"Jan",
            2:"Feb",
            3:"Mar",
            4:"Apr",
            5:"May",
            6:"Jun",
            7:"Jul",
            8:"Aug",
            9:"Sep",
            10:"Oct",
            11:"Nov",
            12:"Dec"
        }
    
    def get_prices(self):
        try:
            r = requests.get("https://www1.nseindia.com/products/dynaContent/equities/indices/historicalindices.jsp?indexType=NIFTY BANK&fromDate=%s&toDate=%s"%(self.from_date,self.to_date),
                            headers=self.headers,verify=False)
            self.construct_csv(self.prices_file,r.text)
            
        except Exception as e:
            print(str(e))
            
    def get_options(self,batch):
        df = pd.read_csv(self.prices_file)
        # Assuming price is 5 digits
        self.strike = int(df["Close"][0]/1000)*1000
        opt_types = ["PE","CE"]
        f_month = self.months[int(self.from_date.split("-")[1])]
        t_month = self.months[int(self.to_date.split("-")[1])]
        
        #Setting up from
        tmp = self.from_date.split("-")
        tmp[1] = f_month
        self.opt_from = "-".join(tmp)
        
        #Setting up to
        tmp = self.to_date.split("-")
        tmp[1] = t_month
        self.opt_to = "-".join(tmp)
        
        for opt_type in opt_types:
            url = "https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?instrumentType=OPTIDX&symbol=BANKNIFTY&expiryDate=%s&optionType=%s&strikePrice=%s&dateRange=&fromDate=%s&toDate=%s&segmentLink=9&symbolCount="%(self.to_date,opt_type,self.strike,self.opt_from,self.opt_to)
            r = requests.get(url,headers=self.headers,verify=False)
            self.construct_csv("banknifty-options-"+opt_type+"-"+str(batch)+".csv",r.text)
        
    
    def construct_csv(self,file_name,data):
        with open(file_name,"w") as f:
            data = "\n".join(BeautifulSoup(data,"html.parser").find("div",{"id":"csvContentDiv"}).text.split(":"))
            f.write(data)
    

In [15]:
# g = GetData("01-06-2022","23-06-2022","test.csv")

In [16]:
# g.get_prices()

In [17]:
# a = "01-06-2022".split("-")
# a[1] = "Jun"

In [18]:
# g.get_options(2)

In [19]:
dates = [
    ["01-01-2019","28-02-2019"],
    ["01-03-2019","25-04-2019"],
    ["01-05-2019","27-06-2019"],
    ["01-07-2019","29-08-2019"],
    ["01-09-2019","31-10-2019"],
    ["01-11-2019","26-12-2019"],
]

In [20]:
# for batch,i in enumerate(dates):
#     batch += 1
#     g = GetData(i[0],i[1],"banknifty-prices-"+str(batch)+".csv")
#     g.get_prices()
#     g.get_options(batch)

In [39]:
class GetStockData(GetData):
    def __init__(self,from_date,to_date,prices_file,stock):
        self.from_date = from_date
        self.errors = []
        self.to_date = to_date
        self.stock = stock
        self.prices_file = prices_file
        self.headers = {
            "Accept":"*/*",
            "Referer":"https://www1.nseindia.com/products/content/derivatives/equities/historical_fo.htm",
            "Accept-Encoding":"gzip, deflate",
            "User-Agent":"Google",
            "Accept-Language":"en-US,en;q=0.9,hi;q=0.8,ru;q=0.7"
        }

        self.proxies = {
            "http":"127.0.0.1:8080",
            "https":"127.0.0.1:8080"
        }
        
        self.months = {
            1:"Jan",
            2:"Feb",
            3:"Mar",
            4:"Apr",
            5:"May",
            6:"Jun",
            7:"Jul",
            8:"Aug",
            9:"Sep",
            10:"Oct",
            11:"Nov",
            12:"Dec"
        }
    
    def get_options(self,batch):
        df = pd.read_csv(self.prices_file)
        # Rounding price to nearest ATM options
        # Minimum 3 digit prices required
        digs = len(str(int(df["Close Price"][0])))
        if digs > 2:
            digs -= 1
        p = df["Close Price"][0]
        self.strike = int(int(p)/(10**(digs-1)))*(10**(digs-1))
        opt_types = ["PE","CE"]
        f_month = self.months[int(self.from_date.split("-")[1])]
        t_month = self.months[int(self.to_date.split("-")[1])]
        
        #Setting up from
        tmp = self.from_date.split("-")
        tmp[1] = f_month
        self.opt_from = "-".join(tmp)
        
        #Setting up to
        tmp = self.to_date.split("-")
        tmp[1] = t_month
        self.opt_to = "-".join(tmp)
        
        for opt_type in opt_types:
            try:
                url = "https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?instrumentType=OPTSTK&symbol=%s&expiryDate=%s&optionType=%s&strikePrice=%s&dateRange=&fromDate=%s&toDate=%s&segmentLink=9&symbolCount="%(self.stock,self.to_date,opt_type,self.strike,self.opt_from,self.opt_to)
                r = requests.get(url,headers=self.headers,verify=False,proxies=self.proxies)
                self.construct_csv("%s-options-"%(self.stock)+opt_type+"-"+str(batch)+".csv",r.text)
            except Exception as e:
                print("Error in batch %d : "%(batch) + str(e))
                self.errors.append(str(batch))
                continue
            
    def get_prices(self):
        try:
            url = "https://www1.nseindia.com/products/dynaContent/common/productsSymbolMapping.jsp?symbol=%s&segmentLink=3&symbolCount=1&series=ALL&dateRange=+&fromDate=%s&toDate=%s&dataType=PRICEVOLUMEDELIVERABLE"%(self.stock,self.from_date,self.to_date)
            r = requests.get(url,
                            headers=self.headers,verify=False,proxies=self.proxies)
            self.construct_csv(self.prices_file,r.text)
            
        except Exception as e:
            print(str(e))

In [40]:
def get_thursdays(mat):
    last = []
    for x in mat:
        for index,val in enumerate(x):
            if(index == 3 and val != 0):
                last.append(val)
    return last[-1]

def generate_full_year(year):
    x = 1
    dates = []
    while(x <= 12):
        start = "01-"+str(x)+"-"+str(year)
        end = str(get_thursdays(calendar.monthcalendar(year,x))) + "-"+ str(x) + "-"+str(year)
        dates.append([start,end])
        x += 1
    return dates

In [41]:
# dates = [
#     ["01-01-2019","31-01-2019"],
#     ["01-02-2019","28-02-2019"],
#     ["01-03-2019","28-03-2019"],
#     ["01-04-2019","25-04-2019"],
#     ["01-05-2019","30-05-2019"],
#     ["01-06-2019","27-06-2019"],
#     ["01-07-2019","25-07-2019"],
#     ["01-08-2019","29-08-2019"],
#     ["01-09-2019","26-09-2019"],
#     ["01-10-2019","31-10-2019"],
#     ["01-11-2019","28-11-2019"],
#     ["01-12-2019","26-12-2019"],
# ]

dates = generate_full_year(2021)
dates

[['01-1-2021', '28-1-2021'],
 ['01-2-2021', '25-2-2021'],
 ['01-3-2021', '25-3-2021'],
 ['01-4-2021', '29-4-2021'],
 ['01-5-2021', '27-5-2021'],
 ['01-6-2021', '24-6-2021'],
 ['01-7-2021', '29-7-2021'],
 ['01-8-2021', '26-8-2021'],
 ['01-9-2021', '30-9-2021'],
 ['01-10-2021', '28-10-2021'],
 ['01-11-2021', '25-11-2021'],
 ['01-12-2021', '30-12-2021']]

In [42]:
for batch,i in enumerate(dates):
    batch += 1
    g = GetStockData(i[0],i[1],"AMBUJACEM-prices-"+str(batch)+".csv","AMBUJACEM")
    g.get_prices()
    g.get_options(batch)
    with open("errors.txt","w") as f:
        f.write(",".join(g.errors))

c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

Error in batch 6 : 'NoneType' object has no attribute 'text'
Error in batch 6 : 'NoneType' object has no attribute 'text'


c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

Error in batch 8 : 'NoneType' object has no attribute 'text'
Error in batch 8 : 'NoneType' object has no attribute 'text'


c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

Error in batch 9 : 'NoneType' object has no attribute 'text'
Error in batch 9 : 'NoneType' object has no attribute 'text'


c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

In [75]:
import calendar

In [82]:
a = calendar.monthcalendar(2022,7)
a

[[0, 0, 0, 0, 1, 2, 3],
 [4, 5, 6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15, 16, 17],
 [18, 19, 20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29, 30, 31]]

In [87]:
last = []
for x in a:
    for index,val in enumerate(x):
        if(index == 3):
            last.append(val)

In [88]:
last

[0, 7, 14, 21, 28]

In [19]:
digs = len(str(int(62.65)))
p =62.65
strike = int(int(p)/(10**(digs-1)))*(10**(digs-1))

In [20]:
strike

60